In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import itertools
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
code_dir = os.path.realpath(os.path.join(os.getcwd(), ".."))
import sys
sys.path.append(code_dir)
from mechaqredo.params import validate_params_dict, default_params_dict
from mechaqredo.sim import run_single_sim,estimate_sensitivity,get_single_derivative
from mechaqredo.generate_scenarios import generate_full_scenario

In [2]:
SIMULATION_LENGHT=365 * 2

sd = {"service_fee_model": 'very good', "price_model": 'optimistic', "n_validators": 'optimistic', "n_trx": 'optimistic'}


params=default_params_dict(SIMULATION_LENGHT)
params_dict = generate_full_scenario(
    params_dict=params, scenarios_dict=sd, forecast_length=SIMULATION_LENGHT
)
df=run_single_sim(SIMULATION_LENGHT,params)
df
df.columns

KeyError: 'optimistic'

In [ ]:
df=get_single_derivative(SIMULATION_LENGHT,with_respect_to='tipping_rate',input_params_dict=params,seed=0)
print('A single derivative')
df

In [ ]:
with_respect_to='staking_renewal_rate'
h=params[with_respect_to]*0.01
params[with_respect_to]  
    


In [ ]:
    seed=0
    np.random.seed(seed)
    df0=run_single_sim(SIMULATION_LENGHT, params)
    np.random.seed(seed)
    params[with_respect_to]+=h
    df1=run_single_sim(SIMULATION_LENGHT, params)
    params[with_respect_to]-=h
    plt.plot((-df0['circ_supply']+df1['circ_supply'])/h)
    

In [ ]:
Ntr=20
tr=np.linspace(0.1,0.95,Ntr)
time=np.arange(SIMULATION_LENGHT)
derivatives=np.zeros((len(tr),len(time)))
import tqdm
for i in tqdm.tqdm(range(Ntr)):
    params=default_params_dict(SIMULATION_LENGHT)
    params_dict = generate_full_scenario(
    params_dict=params, scenarios_dict=sd, forecast_length=SIMULATION_LENGHT)
    
    params['protocol_fee_rate']=0.1*np.random.random()
    params['min_stake_amount']=5000*np.random.random()
    params["min_stake_duration"]= np.random.randint(56)
    params["validator_reward_share"]=np.random.random()
    
    params['staking_renewal_rate']=tr[i]
    df=estimate_sensitivity(SIMULATION_LENGHT,with_respect_to='staking_renewal_rate',input_params_dict=params,N=100)
    derivatives[i]=df['staking_tvl'].values


In [ ]:
TR,TIME=np.meshgrid(tr,time)
p=plt.contourf(TR,TIME,derivatives.T)
plt.colorbar(p)

In [ ]:
QoI='staking_tvl'
params['staking_renewal_rate']=0.1
plt.plot(run_single_sim(SIMULATION_LENGHT,params)[QoI],label=0.1)
params['staking_renewal_rate']=0.8
plt.plot(run_single_sim(SIMULATION_LENGHT,params)[QoI],label=0.8)
params['staking_renewal_rate']=0.9
plt.plot(run_single_sim(SIMULATION_LENGHT,params)[QoI],label=0.9)
plt.legend()
plt.title(QoI)